In [ ]:
# 必要なもの
!pip install transformers==4.57.6
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
# データセットの入力制限
sample_text = dataset["train"][1]["article"][:2000]
# あとから各モデルで生成された要約を格納する
summaries = {}

In [ ]:
# NLTK パッケージ
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

In [ ]:
# NLTK によるピリオドの区別を確認
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

In [ ]:
# ベースラインを用意
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3]) # はじめの 3 文を抜き出す

summaries["baseline"] = three_sentence_summary(sample_text)

In [ ]:
# GPT-2 による結果
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

In [ ]:
# T5 による結果
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))